In [36]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer

In [5]:
spark = SparkSession.builder.appName('airbnb').getOrCreate()

22/04/11 16:18:04 WARN Utils: Your hostname, Charless-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.100 instead (on interface en0)
22/04/11 16:18:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/11 16:18:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Created only one Spark cluster, but when in the cloud can create multiple clusters and instances

In [6]:
spark

### Read in data

In [19]:
df = spark.read.option('header', 'true').csv("datasets/Boston.csv", inferSchema=True)

### Inspect data

In [20]:
df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



In [21]:
type(df)

pyspark.sql.dataframe.DataFrame

In [22]:
df.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [24]:
df.select(['crim', 'zn']).show()

+-------+----+
|   crim|  zn|
+-------+----+
|0.00632|18.0|
|0.02731| 0.0|
|0.02729| 0.0|
|0.03237| 0.0|
|0.06905| 0.0|
|0.02985| 0.0|
|0.08829|12.5|
|0.14455|12.5|
|0.21124|12.5|
|0.17004|12.5|
|0.22489|12.5|
|0.11747|12.5|
|0.09378|12.5|
|0.62976| 0.0|
|0.63796| 0.0|
|0.62739| 0.0|
|1.05393| 0.0|
| 0.7842| 0.0|
|0.80271| 0.0|
| 0.7258| 0.0|
+-------+----+
only showing top 20 rows



In [26]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|             black|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

### Drop null values

In [29]:
df = df.na.drop()

In [30]:
df.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

### Impute new columns

In [37]:
imputer = Imputer(
    inputCols=['crim', 'zn', 'indus'],
    outputCols=["{}_imputed".format(c) for c in ['crim', 'zn', 'indus']]
    ).setStrategy("mean")

In [39]:
imputer.fit(df).transform(df).select(['crim', 'zn', 'indus',
                                     'crim_imputed', 'zn_imputed', 'indus_imputed']).show()

+-------+----+-----+------------+----------+-------------+
|   crim|  zn|indus|crim_imputed|zn_imputed|indus_imputed|
+-------+----+-----+------------+----------+-------------+
|0.00632|18.0| 2.31|     0.00632|      18.0|         2.31|
|0.02731| 0.0| 7.07|     0.02731|       0.0|         7.07|
|0.02729| 0.0| 7.07|     0.02729|       0.0|         7.07|
|0.03237| 0.0| 2.18|     0.03237|       0.0|         2.18|
|0.06905| 0.0| 2.18|     0.06905|       0.0|         2.18|
|0.02985| 0.0| 2.18|     0.02985|       0.0|         2.18|
|0.08829|12.5| 7.87|     0.08829|      12.5|         7.87|
|0.14455|12.5| 7.87|     0.14455|      12.5|         7.87|
|0.21124|12.5| 7.87|     0.21124|      12.5|         7.87|
|0.17004|12.5| 7.87|     0.17004|      12.5|         7.87|
|0.22489|12.5| 7.87|     0.22489|      12.5|         7.87|
|0.11747|12.5| 7.87|     0.11747|      12.5|         7.87|
|0.09378|12.5| 7.87|     0.09378|      12.5|         7.87|
|0.62976| 0.0| 8.14|     0.62976|       0.0|         8.1

### Filter Data

In [46]:
df.filter(df['age'] <= 60).select(['tax', 'age']).show()

+---+----+
|tax| age|
+---+----+
|222|45.8|
|222|54.2|
|222|58.7|
|311|39.0|
|307|56.5|
|307|29.3|
|307|36.6|
|279|41.5|
|279|30.2|
|252|21.8|
|252|15.8|
|233| 2.9|
|233| 6.6|
|233| 6.5|
|233|40.0|
|233|33.8|
|233|33.3|
|243|45.7|
|243|21.1|
|243|21.4|
+---+----+
only showing top 20 rows



In [49]:
df.filter((df['age'] <= 60) & (df['tax'] < 300)).select(['tax', 'age']).show()

+---+----+
|tax| age|
+---+----+
|222|45.8|
|222|54.2|
|222|58.7|
|279|41.5|
|279|30.2|
|252|21.8|
|252|15.8|
|233| 2.9|
|233| 6.6|
|233| 6.5|
|233|40.0|
|233|33.8|
|233|33.3|
|243|45.7|
|243|21.1|
|243|21.4|
|226|21.9|
|256|40.5|
|284|29.2|
|284|47.2|
+---+----+
only showing top 20 rows



### Aggregate

In [52]:
df.select(['tax', 'age', 'rad']).groupBy('rad').mean().show()

+---+------------------+-----------------+--------+
|rad|          avg(tax)|         avg(age)|avg(rad)|
+---+------------------+-----------------+--------+
|  1|            291.45|           45.025|     1.0|
|  6|372.88461538461536|60.14230769230768|     6.0|
|  3|246.07894736842104|49.31052631578947|     3.0|
|  5| 331.8434782608696|69.22347826086958|     5.0|
|  4| 335.9818181818182|60.84181818181818|     4.0|
|  8|            301.25|            67.35|     8.0|
|  7| 304.4117647058824|40.14117647058823|     7.0|
| 24|             666.0|89.80530303030304|    24.0|
|  2| 260.6666666666667|64.77083333333333|     2.0|
+---+------------------+-----------------+--------+



### ML
use rm and lstat to predict medv

In [53]:
from pyspark.ml.feature import VectorAssembler

In [54]:
featureassembler = VectorAssembler(inputCols=['rm', 'lstat'], outputCol = 'independent_featire')

In [57]:
output = featureassembler.transform(df['rm', 'lstat', 'medv'])

In [58]:
output.show()

+-----+-----+----+-------------------+
|   rm|lstat|medv|independent_featire|
+-----+-----+----+-------------------+
|6.575| 4.98|24.0|       [6.575,4.98]|
|6.421| 9.14|21.6|       [6.421,9.14]|
|7.185| 4.03|34.7|       [7.185,4.03]|
|6.998| 2.94|33.4|       [6.998,2.94]|
|7.147| 5.33|36.2|       [7.147,5.33]|
| 6.43| 5.21|28.7|        [6.43,5.21]|
|6.012|12.43|22.9|      [6.012,12.43]|
|6.172|19.15|27.1|      [6.172,19.15]|
|5.631|29.93|16.5|      [5.631,29.93]|
|6.004| 17.1|18.9|       [6.004,17.1]|
|6.377|20.45|15.0|      [6.377,20.45]|
|6.009|13.27|18.9|      [6.009,13.27]|
|5.889|15.71|21.7|      [5.889,15.71]|
|5.949| 8.26|20.4|       [5.949,8.26]|
|6.096|10.26|18.2|      [6.096,10.26]|
|5.834| 8.47|19.9|       [5.834,8.47]|
|5.935| 6.58|23.1|       [5.935,6.58]|
| 5.99|14.67|17.5|       [5.99,14.67]|
|5.456|11.69|20.2|      [5.456,11.69]|
|5.727|11.28|18.2|      [5.727,11.28]|
+-----+-----+----+-------------------+
only showing top 20 rows



In [59]:
data = output.select("independent_featire", 'medv')

In [60]:
data.show()

+-------------------+----+
|independent_featire|medv|
+-------------------+----+
|       [6.575,4.98]|24.0|
|       [6.421,9.14]|21.6|
|       [7.185,4.03]|34.7|
|       [6.998,2.94]|33.4|
|       [7.147,5.33]|36.2|
|        [6.43,5.21]|28.7|
|      [6.012,12.43]|22.9|
|      [6.172,19.15]|27.1|
|      [5.631,29.93]|16.5|
|       [6.004,17.1]|18.9|
|      [6.377,20.45]|15.0|
|      [6.009,13.27]|18.9|
|      [5.889,15.71]|21.7|
|       [5.949,8.26]|20.4|
|      [6.096,10.26]|18.2|
|       [5.834,8.47]|19.9|
|       [5.935,6.58]|23.1|
|       [5.99,14.67]|17.5|
|      [5.456,11.69]|20.2|
|      [5.727,11.28]|18.2|
+-------------------+----+
only showing top 20 rows



In [63]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='independent_featire', labelCol = 'medv')
regressor = regressor.fit(train_data)

22/04/11 17:14:26 WARN Instrumentation: [8246d0ca] regParam is zero, which might cause numerical instability and overfitting.
22/04/11 17:14:26 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/11 17:14:26 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/11 17:14:26 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [64]:
regressor.coefficients

DenseVector([4.4746, -0.7141])

In [65]:
predictions = regressor.evaluate(test_data)

In [66]:
predictions.predictions.show()

+-------------------+----+-------------------+
|independent_featire|medv|         prediction|
+-------------------+----+-------------------+
|      [4.628,34.37]|17.9|-0.5736947753425983|
|      [4.652,28.28]|10.5|  3.882476734846077|
|       [4.88,30.62]|10.2|  3.231728991362056|
|      [5.012,12.12]|15.3|  17.03296012082872|
|      [5.019,34.41]|14.4| 1.1473164798963817|
|      [5.093,29.68]| 8.1| 4.8560624775664145|
|      [5.155,20.08]|16.3| 11.988709507575262|
|      [5.272,16.14]|13.1| 15.325736422913831|
|      [5.304,24.91]|12.0|  9.206394224275638|
|      [5.304,26.64]|10.4|  7.971026297268771|
|      [5.362,10.19]|20.8|  19.97726058766563|
|      [5.399,30.81]|14.4|  5.418378209475897|
|      [5.404,23.98]|19.3| 10.317955299570754|
|      [5.412,29.55]|23.7|  6.376295950220054|
|      [5.414,23.97]| 7.0| 10.369842310024215|
|      [5.453,30.59]| 5.0|  5.817106260076686|
|      [5.468,26.42]|15.6|  8.861962057094171|
|       [5.52,24.56]|12.3| 10.422841108000384|
|      [5.531

/Users/charlesreinertson/Desktop/School/grad/DATA514/spark_database/env_spark/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Next step put this in the cloud! Multiple clusters (databricks?)